# Optimal Control

In [1]:
import os
import numpy as np
from numpy.testing import assert_array_equal, assert_array_almost_equal
import matplotlib.pyplot as plt
from casadi import (
    MX, DM, Function, Opti,
    vec, vcat, vertcat, horzcat, sum1, sum2, triu,
    gradient, jacobian, hessian
)

In [2]:
data_dir = 'data'

## 1. Solve Multiple Shooting Problem

In [3]:
# State transition function
uk = MX.sym('uk')
xk = MX.sym('xk')
F = Function('F', [xk, uk], [xk ** 2 + uk], ['xk', 'uk'], ['xkp1'])
F

Function(F:(xk,uk)->(xkp1) MXFunction)

Discrete optimal control problem:

$$
\DeclareMathOperator*{\minimize}{minimize}
\begin{aligned}
\minimize_{x_1,x_2,...,x_{N+1},u_1,u_2,...,u_N} \quad & \sum_{k=1}^{N}{u_k^2} + \sum_{k=1}^{N+1}{x_k^2} \\
\text{subject to} \quad & F(x_k,u_k) = x_{k+1}, \quad k=1, ..., N \\
                  & x_1 = 2 \\
                  & x_{N+1} = 3 \\
                  & x_k \ge 0, \quad k=1, ..., N+1 \\
\end{aligned}
$$


In [4]:
opti = Opti()
N = 4
X = opti.variable(N + 1)
U = opti.variable(N)

# Define objective function
f = sum1(X ** 2) + sum1(U ** 2)

# Define constraints
for k in range(1, N + 1):
    opti.subject_to(X[k] == F(X[k-1], U[k-1]))
opti.subject_to(X[0] == 2)
opti.subject_to(X[-1] == 3)
opti.subject_to(X >= 0)

opti.minimize(f)

In [5]:
opti.solver('ipopt')
sol = opti.solve()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:       14
Number of nonzeros in inequality constraint Jacobian.:        5
Number of nonzeros in Lagrangian Hessian.............:        9

Total number of variables............................:        9
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        6
Total number of inequality c

In [6]:
sol.value(U)

array([-2.70380638, -0.54297936,  0.26125189,  0.58403971])

In [7]:
assert_array_almost_equal(sol.value(U), [-2.7038, -0.5430, 0.2613, 0.5840], decimal=4)

## 2. Adapt to single-shooting problem

In [8]:
opti = Opti()

# Objective function
N = 4
x0 = opti.variable()
U = opti.variable(N)
X = [x0]
for k in range(1, N + 1):
    X.append(F(X[k-1], U[k-1]))
X = vertcat(*X)
f = sum1(X ** 2) + sum1(U ** 2)

# Define constraints
opti.subject_to(X[0] == 2)
opti.subject_to(X[-1] == 3)
opti.subject_to(X >= 0)

opti.minimize(f)

In [9]:
opti.solver('ipopt')
sol = opti.solve()

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        6
Number of nonzeros in inequality constraint Jacobian.:       15
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        5
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 3.00e+00 5.00e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [10]:
sol.value(U)

array([-2.70380638, -0.54297936,  0.26125189,  0.58403971])

In [11]:
assert_array_almost_equal(sol.value(U), [-2.7038, -0.5430, 0.2613, 0.5840], decimal=4)

## 3. Plot sparsities of the Hessian